# Indicadores de tendencia.

Analizando series temporales.  https://quantspace.es/2019/11/22/analizando-series-temporales-con-python/#comment-7872

In [46]:
import pandas as pd
import numpy as np
import yfinance as yf
from termcolor import colored

class CalculaIndicadoresTendencia():
    def __init__(self, ticker, start_date, end_date): 
        """
        Inicializa la clase CalculaIndicadoresTendencia.

        Args:
        - ticker: Símbolo del activo.
        - nombre: Nombre de la columna en datos que contiene las cotizaciones.
        - short_name: Nombre corto del activo.
        - start_date: Fecha de inicio del período de análisis.
        - end_date: Fecha de fin del período de análisis.
        """
        self.ticker = ticker
        self.diagnostico = ""
        self.cuenta_menor = 0 
        self.cuenta_mayor = 0
        self.media_subidas = 0
        self.media_bajadas = 0
        self.cotiz_max = 0
        self.cotiz_min = 0
        self.fecha_cotiz_max = None
        self.fecha_cotiz_min = None
        self.start_date = start_date
        self.end_date = end_date
        self.historical_quotes = None
        self.close_df = None
        self.name = None

    def get_quotes(self):
        """
        Obtiene los datos de cotización de un ticker en un rango de fechas.

        Returns:
        - historical_quotes: DataFrame con los datos históricos de cotización.
        - close_df: DataFrame con los precios de cierre.
        - name: Nombre del activo.
        """
        yfObj = yf.Ticker(self.ticker)
        self.name = yfObj.info['shortName']
        self.historical_quotes = yf.download(self.ticker, start=self.start_date, end=self.end_date)
        self.close_df = self.historical_quotes[['Close']].rename(columns={'Close': self.name})
        #print (close_df)
        return self.historical_quotes, self.close_df, self.name

    def calculate_variations(self):
        """
        Calcula las variaciones, log-retornos y retornos acumulados.
        """
        self.close_df["variations"] = self.close_df[self.name].diff()
        self.close_df["log_returns"] = np.log(self.close_df[self.name]/self.close_df[self.name].shift(1)).dropna() 
        self.close_df["cum_returns"]  = self.close_df["log_returns"].cumsum().apply(np.exp) 
        return self.close_df
    
    def cuenta_variaciones(self):
        """
        Cuenta las variaciones positivas y negativas en los log-retornos diarios.
        """
        if self.close_df is None:
            print("Error: No se han calculado las variaciones. Llama al método calculate_variations() primero.")
            return
        
        self.df_val_macd = self.close_df.log_returns * 100  # Calcula los valores en porcentaje
        for i in self.df_val_macd:
            if i >= 0:
                self.cuenta_mayor += 1
            else:               
                self.cuenta_menor += 1
    
    def calc_cotiz_MediaMaxMin(self):
        """
        Calcula la cotización media, máxima, mínima, y medias de subidas y bajadas.
        """
        if self.close_df is None:
            print("Error: No se han calculado las variaciones. Llama al método calculate_variations() primero.")
            return
        
        # Cotización media, máxima y mínima en el período
        self.cotizacion_media = self.close_df[self.name].mean()        
        self.cotiz_max = self.close_df[self.name].max()
        self.cotiz_min = self.close_df[self.name].min()
        
        # Índice de la fila con el valor máximo y mínimo de la cotización
        self.fecha_cotiz_max = self.close_df[self.name].idxmax()
        self.fecha_cotiz_min = self.close_df[self.name].idxmin()        

        # Media de subidas y bajadas               
        self.df_val_macd1 = self.df_val_macd[self.df_val_macd >= 0] 
        self.media_subidas = self.df_val_macd1.mean()        
        self.df_val_macd2 = self.df_val_macd[self.df_val_macd < 0] 
        self.media_bajadas = self.df_val_macd2.mean()
                   
    def imprime_indicadores(self):
        """
        Imprime los indicadores de tendencia calculados.
        """
        if self.close_df is None:
            print("Error: No se han calculado los indicadores. Llama al método calculate_variations() primero.")
            return
        
        print(colored("\n*****************************************************************************************************************", 'blue', attrs=['bold']))
        print(colored(f"\nINDICADORES DE TENDENCIA  ", 'blue', attrs=['bold']))  
        print(colored(f"\nTendencia de las cotizaciones en el Período del {self.start_date} al {self.end_date}", 'blue', attrs=['bold']))
        print(colored("\n*****************************************************************************************************************", 'blue', attrs=['bold']))
        print ("\nNúmero de variaciones >= 0 :", self.cuenta_mayor)
        print ("\nNúmero de variaciones < 0 :", self.cuenta_menor)
        print ("\nCotización media en el período :", round(self.cotizacion_media, 2))
        print ("\nCotizacion máxima en el período :", round(self.cotiz_max, 2), self.fecha_cotiz_max )
        print ("\nCotizacion mínima en el período :", round(self.cotiz_min,2), self.fecha_cotiz_min )
        print ("\nPromedio de las subidas % :", round(self.media_subidas,3))
        print ("\nPromedio de las bajadas % :", round(self.media_bajadas,3))
        print ("")

    def semaforo(self):
            """
            Realiza un análisis de semáforo de la evolución de la cotización.
            """
            if self.close_df is None:
                print("Error: No se han calculado los indicadores. Llama al método calculate_variations() primero.")
                return
    
            dif20 = 0
            dif200 = 0
            diagnostico = ""
    
            media_20dias = self.close_df.iloc[-21:-1, 0].mean()
            media_200dias = self.close_df.iloc[-201:-1, 0].mean()
            ultima_cotizacion = self.close_df.iloc[-1, 0]
    
            dif20 = ultima_cotizacion - media_20dias     
            dif200 = ultima_cotizacion - media_200dias
    
            if dif20 > 0:
                if dif200 > 0:
                    diagnostico = "\nTendencia alcista"
                elif  dif200 < 0:
                    diagnostico = "\nTendencia neutra"
    
            elif  dif20 <= 0:
                if dif200 <= 0:
                    diagnostico = "\nTendencia bajista"
                elif  dif200 > 0:
                    diagnostico = "Tendencia neutra"
            else:
                diagnostico = "\nTendencia neutra"
    
            print ("\nSemáforo de la evolución")
            print ("\nEl valor actual supera la media de las últimas 20 cotizaciones :\n", round(dif20, 2))
            print ("\nEl valor actual supera la media de las últimas 200 cotizaciones :\n", round(dif200, 2))
            print ("\nDiagnóstico :", diagnostico)

ticker = '^IBEX'
start_date = "2010-01-01"
end_date = "2024-3-15"

indicadores_tendencia = CalculaIndicadoresTendencia(ticker, start_date, end_date)
historical_quotes, close_df, name = indicadores_tendencia.get_quotes()

# Calcula las variaciones.
indicadores_tendencia.calculate_variations()

# Cálculo de los indicadores
indicadores_tendencia.cuenta_variaciones()    
indicadores_tendencia.calc_cotiz_MediaMaxMin()    

# Imprime los indicadores y el semáforo
indicadores_tendencia.imprime_indicadores()  
indicadores_tendencia.semaforo()  


[*********************100%%**********************]  1 of 1 completed


*****************************************************************************************************************

INDICADORES DE TENDENCIA  

Tendencia de las cotizaciones en el Período del 2010-01-01 al 2024-3-15

*****************************************************************************************************************

Número de variaciones >= 0 : 1876

Número de variaciones < 0 : 1757

Cotización media en el período : 9230.74

Cotizacion máxima en el período : 12222.5 2010-01-06 00:00:00

Cotizacion mínima en el período : 5956.3 2012-07-24 00:00:00

Promedio de las subidas % : 0.925

Promedio de las bajadas % : -0.997


Semáforo de la evolución

El valor actual supera la media de las últimas 20 cotizaciones :
 348.26

El valor actual supera la media de las últimas 200 cotizaciones :
 841.76

Diagnóstico : 
Tendencia alcista
